<a href="https://colab.research.google.com/github/Akankshakumbhar/CODINGSAMURAI/blob/master/Semtimentanaylsis%20twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sentiment anaylsis (twitter)


In [2]:
import pandas as pd
import numpy as np

In [3]:
import seaborn as sn

In [4]:
import matplotlib.pyplot as plt

In [12]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kazanova/sentiment140")

print("Path to dataset files:", path)

100%|██████████| 80.9M/80.9M [00:00<00:00, 185MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/kazanova/sentiment140/versions/2


In [13]:
! pip install kaggle

In [14]:
#configure the path of kaggle .json file

In [20]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [22]:
# Step 1: Upload kaggle.json
from google.colab import files

print("Please upload your kaggle.json file.")
files.upload()  # Select kaggle.json when prompted

# Step 2: Move kaggle.json to the correct directory
import os

# Create the .kaggle directory
!mkdir -p ~/.kaggle

# Move the uploaded file to the .kaggle directory
!cp kaggle.json ~/.kaggle/

# Set correct permissions for kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

# Step 3: Test Kaggle authentication
try:
    print("Testing Kaggle authentication...")
    !kaggle datasets list
    print("Kaggle is successfully authenticated!")
except Exception as e:
    print(f"Error: {e}")


Please upload your kaggle.json file.


Saving kaggle (1).json to kaggle (1).json
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Testing Kaggle authentication...
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/
Kaggle is successfully authenticated!


In [19]:
import os
print(os.listdir())


['.config', 'kaggle .json', '.ipynb_checkpoints', 'sample_data']


In [23]:
!kaggle  datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 91% 74.0M/80.9M [00:00<00:00, 256MB/s]
100% 80.9M/80.9M [00:00<00:00, 251MB/s]


In [25]:
from zipfile import ZipFile

In [26]:
datafile='/content/sentiment140.zip'

In [27]:
with ZipFile(datafile,'r')as zip:
  zip.extractall()

In [30]:
import pandas as pd

In [40]:
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [43]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [44]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [45]:
data=pd.read_csv('/content/twitter.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [46]:
data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [47]:
data.shape

(1599999, 6)

In [48]:
column_name=['target','id','date','flag','user','text']
data=pd.read_csv('/content/twitter.1600000.processed.noemoticon.csv',names=column_name,encoding='ISO-8859-1')

In [49]:
data.head(4)

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire


In [51]:
data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [55]:
data['target'].value_counts()

,count
target,
0,800000
4,800000


In [58]:
data.replace({'target':{4:1}},inplace=True)

In [59]:
data.shape

(1600000, 6)

In [60]:
data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [61]:
#negavtive tweet 0  positive tweet  1

In [64]:
# stemming ( the process of reducing thw word into rootword )
#eg actor,actress, acting so root word is actor

In [65]:
port_stem=PorterStemmer()

In [68]:
def stemming(content):
  stemmedconent=re.sub('[^a-zA-Z]',' ',content)
  stemmedconent=stemmedconent.lower()
  stemmedconent=stemmedconent.split()
  stemmedconent=[port_stem.stem(word) for word in stemmedconent if not word in stopwords.words('english')]
  stemmedconent=" ".join(stemmedconent)
  return stemmedconent

In [75]:
data['stemmedconent']=data['text'].apply(stemming)

## Importing the libraries

## Importing the dataset

## Splitting the dataset into the Training set and Test set

## Training the Simple Linear Regression model on the Training set

## Predicting the Test set results

## Visualising the Training set results

## Visualising the Test set results

## Let's predict a salary of an employee with number of years of experience